In [ ]:
# src/style_transfer.py
"""
Squelette pour Neural Style Transfer (préparation VGG16 + extractor).

Remarques:
 - Fournir deux images (content et style) dans le dossier data/
 - Pré-traiter les images selon keras.applications.vgg16.preprocess_input
 - Implémenter ensuite la boucle d'optimisation (ex: tf.GradientTape) pour
   minimiser content_loss + style_loss.
"""

import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image

# Layers to use
CONTENT_LAYERS = ['block5_conv2']
STYLE_LAYERS = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

def load_and_process_image(path, target_size=(512, 512)):
    img = Image.open(path).convert('RGB').resize(target_size)
    arr = np.array(img).astype('float32')
    arr = np.expand_dims(arr, axis=0)
    arr = keras.applications.vgg16.preprocess_input(arr)
    return arr

def deprocess_image(processed_img):
    # reverse of preprocess_input for VGG16 (approximation)
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]  # BGR -> RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def create_extractor():
    vgg = keras.applications.VGG16(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in (STYLE_LAYERS + CONTENT_LAYERS)]
    model = keras.Model(inputs=vgg.input, outputs=outputs)
    return model

# Exemple d'utilisation:
if __name__ == "__main__":
    extractor = create_extractor()
    print("Extractor created. Output tensors:")
    for o in extractor.outputs:
        print(o)
    # A partir d'ici: charger content_image et style_image, extraire features,
    # définir content_loss et style_loss, puis optimiser l'image cible.


 3981312/58889256 ━━━━━━━━━━━━━━━━━━━━ 27:14 30us/step